<a href="https://colab.research.google.com/github/yerimming/gachon/blob/main/CV_AL_test15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.   optimizer로 RMSGrad 사용
2.   lr = 0.001
3.  CosineAnnealingLR 스케줄러 사용




In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
!pip uninstall adabelief-pytorch

Found existing installation: adabelief-pytorch 0.2.1
Uninstalling adabelief-pytorch-0.2.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/adabelief_pytorch-0.2.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/adabelief_pytorch/*
Proceed (Y/n)? y
  Successfully uninstalled adabelief-pytorch-0.2.1


In [2]:
!pip install adabelief-pytorch

In [3]:
import os
import time

import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from adabelief_pytorch import AdaBelief

# 스케줄러 사용
from torch.optim.lr_scheduler import CosineAnnealingLR

import re

from PIL import Image

### GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


In [4]:
### Custom Dataset ###
class CUB2011(Dataset):
  def __init__(self, transform, mode='train'):
    self.transform = transform
    self.mode = mode

    if self.mode == 'train':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/CUB_200_2011_repackage_class50/datasets/train')
    elif self.mode == 'valid':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/CUB_200_2011_repackage_class50/datasets/valid')
    elif self.mode == 'test':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/CUB_200_2011_repackage_class50/datasets/test')

  def __len__(self):
    return len(self.image_folder)

  def __getitem__(self, idx):
    img_path = self.image_folder[idx]
    img = Image.open(os.path.join('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/CUB_200_2011_repackage_class50/datasets', self.mode, img_path)).convert('RGB')
    img = self.transform(img)
    label = img_path.split('_')[-1].split('.')[0]
    label = re.sub(r'\([^)]*\)', '', label)
    label = int(label)
    return (img, label)

In [5]:
### Data Preprocessing ###

transforms_train = transforms.Compose([transforms.Resize((448,448)),
                                       transforms.ToTensor()])
transforms_valtest = transforms.Compose([transforms.Resize((448,448)),
                                         transforms.ToTensor()])

BATCH_SIZE = 32
train_set = CUB2011(mode='train',
                    transform=transforms_train)
# transforms_train을 transforms_valtest로 변경
val_set = CUB2011(mode='valid',
                  transform=transforms_valtest)
test_set = CUB2011(mode='test',
                  transform=transforms_valtest)

print('Num of each dataset: ',len(train_set),len(val_set),len(test_set))

train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=False)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=False)

print("Loaded dataloader")

Num of each dataset:  2362 296 298
Loaded dataloader


In [7]:
!pip install --upgrade adabelief-pytorch

In [11]:
### Model / Optimizer ###
EPOCH = 30
lr = 0.001

model = models.resnet18(pretrained=True)

### Tranfer Learning ###
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,50)
model.to(DEVICE)

optimizer = AdaBelief(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-16, weight_decouple=True, rectify=True)

# 코사인 앤니얼링 스케줄러 초기화
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0.001)

print("Created a learning model and optimizer")

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
Created a learning model and optimizer


In [12]:
### Train/Evaluation ###
def train(model,train_loader,optimizer,epoch):
  model.train()
  for i,(image,target) in enumerate(train_loader):
    image,target = image.to(DEVICE),target.to(DEVICE)
    output = model(image)
    optimizer.zero_grad()
    # loss func을 어떤 것을 사용할 것인지?
    train_loss = F.cross_entropy(output,target).to(DEVICE)

    train_loss.backward()
    optimizer.step()

    # 스케줄러 업데이트
    scheduler.step()

    if i%10 ==0:
      print(
          f'Train Epoch: {epoch} [{i}/{len(train_loader)}]\tloss: {train_loss.item():6f}')

  return train_loss

def evaluate(model,val_loader):
  model.eval()
  eval_loss = 0
  correct = 0
  with torch.no_grad():
    for i,(image,target) in enumerate(val_loader):
      image,target = image.to(DEVICE),target.to(DEVICE)
      output = model(image)

      eval_loss += F.cross_entropy(output,target, reduction='sum').item()
      pred = output.max(1,keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

  eval_loss /= len(val_loader.dataset)
  eval_accuracy = 100*correct / len(val_loader.dataset)
  return eval_loss,eval_accuracy

In [13]:
### Main ###
start = time.time()
best = 0
for epoch in range(EPOCH):
  train_loss = train(model,train_loader,optimizer,epoch)
  val_loss,val_accuracy = evaluate(model,val_loader)

  # Save best model
  if val_accuracy > best:
    best = val_accuracy
    torch.save(model.state_dict(),"./best_model.pth")

  print(f"[{epoch}]Validation Loss: {val_loss:.4f},Accuracy: {val_accuracy:.4f}%")

# Test result
test_loss,test_accuracy = evaluate(model,test_loader)
print(f'[FINAL] Test Loss: {test_loss:.4f},Accuracy: {test_accuracy:.4f}%')

end = time.time()
elasped_time = end - start

print("Best Accuracy: ",best)
print(
    f"Elasped Time: {int(elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")
print(
    f"time: {int(elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")

Train Epoch: 0 [0/74]	loss: 3.978997
Train Epoch: 0 [10/74]	loss: 3.923540
Train Epoch: 0 [20/74]	loss: 3.749312
Train Epoch: 0 [30/74]	loss: 3.431151
Train Epoch: 0 [40/74]	loss: 3.037728
Train Epoch: 0 [50/74]	loss: 2.336459
Train Epoch: 0 [60/74]	loss: 2.410351
Train Epoch: 0 [70/74]	loss: 2.032878
[0]Validation Loss: 1.7810,Accuracy: 70.2703%
Train Epoch: 1 [0/74]	loss: 1.683001
Train Epoch: 1 [10/74]	loss: 1.354242
Train Epoch: 1 [20/74]	loss: 1.322171
Train Epoch: 1 [30/74]	loss: 1.144216
Train Epoch: 1 [40/74]	loss: 1.064530
Train Epoch: 1 [50/74]	loss: 0.853999
Train Epoch: 1 [60/74]	loss: 0.993312
Train Epoch: 1 [70/74]	loss: 0.872772
[1]Validation Loss: 0.8671,Accuracy: 81.0811%
Train Epoch: 2 [0/74]	loss: 0.553997
Train Epoch: 2 [10/74]	loss: 0.519001
Train Epoch: 2 [20/74]	loss: 0.462329
Train Epoch: 2 [30/74]	loss: 0.395640
Train Epoch: 2 [40/74]	loss: 0.309551
Train Epoch: 2 [50/74]	loss: 0.335176
Train Epoch: 2 [60/74]	loss: 0.434636
Train Epoch: 2 [70/74]	loss: 0.337136